In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Weekly Meeting Example .mp4 to Weekly Meeting Example .mp4


In [ ]:
ls

 sample_data/  'Weekly Meeting Example .mp4'


In [ ]:
from transformers import pipeline,AutoModelForCausalLM, AutoTokenizer
import torch
from IPython.display import  clear_output


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import subprocess

def convert_mp4_to_mp3(input_file, output_file):
    """Convert an MP4 file to MP3 using ffmpeg."""
    try:
        command = [
            "ffmpeg",
            "-i", input_file,          # Input file
            "-vn",                     # Disable video
            "-acodec", "libmp3lame",   # Use MP3 codec
            "-b:a", "192k",            # MP3 bitrate
            output_file                # Output file
        ]
        subprocess.run(command, check=True)
        print(f"Conversion successful: {output_file}")
    except subprocess.CalledProcessError as e:
        print("Error during conversion:", e)

# Run conversion on your newly uploaded file
convert_mp4_to_mp3("Weekly Meeting Example .mp4", "meeting.mp3")


Conversion successful: meeting.mp3


In [ ]:
pipe  = pipeline("automatic-speech-recognition",
                    "openai/whisper-small",
                    chunk_length_s=30,
                    stride_length_s=5,
                    return_timestamps=True,
                    device=device,
                    generate_kwargs = {"language": 'english', "task": "transcribe"})

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
transcription = pipe("meeting.mp3" )
#Once the transcription is complete, we’ll format the text with timestamps for better readability.
formatted_lyrics = ""
for line in transcription['chunks']:
    text = line["text"]
    ts = line["timestamp"]
    formatted_lyrics += f"{ts}-->{text}\n"

print(formatted_lyrics.strip())

(0.0, 85.04)--> Hello, everyone. Thank you guys for coming to our weekly student success meeting, and let's just get started. So I have our list of chronically absent students here, and I've been noticing a troubling trend. A lot of students are skipping on Fridays. Does anyone have any idea what's going on? I've heard some of my mentees talking about how it's really hard to get out of bed on Fridays. It might be good if we did something like a pancake breakfast to encourage them to come. I think that's a great idea. Let's try that next week. It might also be because a lot of students have been getting sick now that it's getting colder outside. I've had a number of students come by my office with symptoms like sniffling and coughing. We should put up posters with tips for not getting sick since it's almost flu season. Like, you know, wash your hands after the bathroom, stuff like that. I think that that's a good idea and it'll be a good reminder for the teachers as well. So one other t

In [ ]:
# Let’s also save the transcription to a text file so we can use it later!"
with open("transcription.txt", "w", encoding="utf-8") as file:
    file.write(formatted_lyrics.strip())

print("Transcription saved to transcription.txt")

Transcription saved to transcription.txt


In [ ]:
DEFAULT_MODEL = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL)
tokenizer.pad_token_id = tokenizer.eos_token_id


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
conversation = [
    {
        "role": "system",
        "content": (
            "You are an expert meeting summarizer. Your job is to read the user's full meeting "
            "transcript and produce a clear, structured, accurate summary.\n\n"
            "The summary should include:\n"
            "- Main topics discussed\n"
            "- Key decisions made\n"
            "- Action items (with who will do what)\n"
            "- Important concerns or issues raised\n"
            "- Any follow-ups required\n\n"
            "DO NOT add any extra topics. ONLY use information from the transcript."
        )
    },
    {
        "role": "user",
        "content": formatted_lyrics.strip()
    }
]


prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# print(prompt)

with torch.no_grad():
    output = model.generate(
        **inputs,
        do_sample=True,
        max_new_tokens=1000
    )

processed_text = tokenizer.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)

print(processed_text)

assistant
### Summary

**Main Topics Discussed:**
1. Chronic absenteeism, particularly Friday absences.
2. Students' health issues due to colder weather.
3. Student John Smith’s unexcused absences (7 days in November).
4. Family caregiving responsibilities affecting John Smith.

**Key Decisions Made:**
1. Organize a pancake breakfast next week to encourage attendance.
2. Post flu prevention tips around campus.
3. John Smith will meet with the guidance counselor to discuss his situation.
4. The speaker will offer support in finding free or low-cost community resources for John Smith’s family.

**Action Items:**
- Organize the pancake breakfast next week.
- Create and post flu prevention tips.
- Arrange a meeting between John Smith and the guidance counselor.
- Identify and provide John Smith with community resources.

**Important Concerns or Issues Raised:**
- Students skipping classes on Fridays.
- Students getting sick due to changing weather conditions.
- John Smith missing multiple 

In [ ]:
print(formatted_lyrics)


(0.0, 85.04)--> Hello, everyone. Thank you guys for coming to our weekly student success meeting, and let's just get started. So I have our list of chronically absent students here, and I've been noticing a troubling trend. A lot of students are skipping on Fridays. Does anyone have any idea what's going on? I've heard some of my mentees talking about how it's really hard to get out of bed on Fridays. It might be good if we did something like a pancake breakfast to encourage them to come. I think that's a great idea. Let's try that next week. It might also be because a lot of students have been getting sick now that it's getting colder outside. I've had a number of students come by my office with symptoms like sniffling and coughing. We should put up posters with tips for not getting sick since it's almost flu season. Like, you know, wash your hands after the bathroom, stuff like that. I think that that's a good idea and it'll be a good reminder for the teachers as well. So one other t